In [1]:
# base path to YOLO directory
MODEL_PATH = "yolo-coco"

# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# boolean indicating if NVIDIA CUDA GPU should be used
USE_GPU = False

# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50

In [2]:

def detectPeople(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter detections by (1) ensuring that the object
			# detected was a person and (2) that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	# return the list of results
	return results

In [3]:


from scipy.spatial import distance as dist
import numpy as np
from tqdm.notebook import tqdm
import argparse
import imutils
import cv2
import os

In [4]:
videoFile = r'pedestrians.mp4'
configPath = r'yolo-coco/yolov3.cfg'
weightsPath = r'yolo-coco/yolov3.weights'
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [5]:
previewWhileProcess = False
saveLocally = True
writer = None

In [6]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print("we are going to use", ln)

we are going to use ['yolo_82', 'yolo_94', 'yolo_106']


In [7]:

cam = cv2.VideoCapture(videoFile)
fps = int(cam.get(cv2.CAP_PROP_FPS))
totalFPS = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))
computedFps = 0
print(fps, totalFPS)

25 531


In [8]:
LABELS = []
with open("yolo-coco/coco.names", 'rt') as f:
    LABELS = f.read().rstrip("\n").split('\n')
print("Number of objects Model hold: ", len(LABELS))
print("We gonna use person Only index of: ",LABELS.index("person"))

Number of objects Model hold:  80
We gonna use person Only index of:  0


In [9]:
color_violation = (0,0, 255)
color_safe = (0,255,0)

In [10]:
print("[INFO] init Process..")
pbar = tqdm(total=totalFPS, desc="Progress") #progressBar
while True:
    success, frame = cam.read()
    
    if not success:
        print("[INFO] EOF Process..")
        break        
    
    frame = imutils.resize(frame, width=700)
    results = detectPeople(frame, net, ln, personIdx=LABELS.index('person'))


    voilation = set()

    if len(results) >=2:
        centroids = np.array([r[2] for r in results])
        distanceMap = dist.cdist(centroids, centroids, metric = "euclidean")

#   print(distanceMap.shape)
        for i in range(distanceMap.shape[0]):
            for j in range(i+1, distanceMap.shape[1]):

                #if distance between 2person is less minDistance
                if distanceMap[i, j] < MIN_DISTANCE:
                    voilation.add(i)
                    voilation.add(j)


        #Loop over results and make visualizations
    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY)  = bbox
        (cX, cY) = centroid

        color = color_safe

        if i in voilation:
            color = color_violation

        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2) #bounding box
        cv2.circle(frame, (cX, cY), 5, color, 1)
    text = "Social Distancing Violations: {}".format(len(voilation))
    cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

    #should be display
    if previewWhileProcess:
        cv2.imshow("Social Distance", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    
    #init save file
    if saveLocally and writer is None:
#         print(size)
        fourcc =cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('output.avi', fourcc, 20, (frame.shape[1], frame.shape[0]), True) 
    
    if writer is not None:
        writer.write(frame)
        
    #ProgressBar 
    computedFps +=1
    pbar.update(1)

pbar.close()
writer.release()     
cv2.destroyAllWindows()

[INFO] init Process..


Progress:   0%|          | 0/531 [00:00<?, ?it/s]

[INFO] EOF Process..
